# SSD300 Training Tutorial

This tutorial explains how to train an SSD300 on the Pascal VOC datasets. The preset parameters reproduce the training of the original SSD300 "07+12" model. Training SSD512 works simiarly, so there's no extra tutorial for that. The same goes for training on other datasets.

You can find a summary of a full training here to get an impression of what it should look like:
[SSD300 "07+12" training summary](https://github.com/pierluigiferrari/ssd_keras/blob/master/training_summaries/ssd300_pascal_07%2B12_training_summary.md)

In [1]:
from keras.optimizers import Adam, SGD
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, TensorBoard
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Model
from matplotlib import pyplot as plt
from keras.preprocessing import image
from imageio import imread

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss_mod import SSDLoss
from keras_loss_function.keras_ssd_loss_proj import SSDLoss_proj

from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder_mod import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize_Modified
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels_Modified
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation_modified
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
import random
np.set_printoptions(precision=20)
import tensorflow as tf
np.random.seed(1337)
%matplotlib inline

Using TensorFlow backend.


## 0. Preliminary note

All places in the code where you need to make any changes are marked `TODO` and explained accordingly. All code cells that don't contain `TODO` markers just need to be executed.

## 1. Set the model configuration parameters


In [2]:
img_height = 300 # Height of the model input images
img_width = 600 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True            # print(y_encoded)

steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

## 2. Build or load the model

You will want to execute either of the two code cells in the subsequent two sub-sections, not both.

In [3]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
weights_path = 'weights/VGG_ILSVRC_16_layers_fc_reduced.h5'

model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.


/scratch/gits/ssd_keras_double/models/keras_ssd300.py:612: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  model = Model(input=[x,geo_1,geo_2],output=[predictions, predictions_proj])


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 600, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 300, 600, 3)  0                                            
__________________________________________________________________________________________________
identity_layer__1 (Lambda)      (None, 300, 600, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
identity_layer__2 (Lambda)      (None, 300, 600, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
input_mean

def Accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for
    multiclass classification problems.
    '''
    return K.mean(K.equal(K.argmax(y_true[:,:,:-4], axis=-1),
                  K.argmax(y_pred[:,:,:-4], axis=-1)))

def proj(y_pred, y_true):
    #add to gt
    acc = tf.constant(0)
    acc = K.mean(K.equal(K.argmax(y_true[:,:,18:-4], axis=-1), K.argmax(y_pred[:,:,18:-4], axis=-1)))
    return acc

def notproj(y_pred, y_true):
    #add to gt
    acc = tf.constant(0,dtype=tf.float32)
    return acc

def Accuracy_proj(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for
    multiclass classification problems.
    '''
    result = tf.cond(tf.equal(tf.shape(y_true)[2],36), lambda: proj(y_pred, y_true), lambda: notproj(y_pred, y_true))
    return result

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss1 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss2 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss3 = SSDLoss_proj(neg_pos_ratio=3, alpha=1.0)
ssd_loss4 = SSDLoss_proj(neg_pos_ratio=3, alpha=1.0)

losses = {
    "predictions_1": ssd_loss1.compute_loss,
    "predictions_2": ssd_loss2.compute_loss,
    "predictions_1_proj": ssd_loss3.compute_loss,
    "predictions_2_proj": ssd_loss4.compute_loss

}
lossWeights = {"predictions_1": 1.0,"predictions_2": 1.0,"predictions_1_proj": 1.0,"predictions_2_proj": 1.0}

model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights, metrics=[Accuracy, Accuracy_proj]) 
# model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights) 


In [5]:
def Accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for
    multiclass classification problems.
    '''
    return K.mean(K.equal(K.argmax(y_true[:,:,:-6], axis=-1),
                  K.argmax(y_pred[:,:,:-6], axis=-1)))

def proj(y_pred, y_true):
    #add to gt
    acc = tf.constant(0)
    acc = K.mean(K.equal(K.argmax(y_true[:,:,18:-4], axis=-1), K.argmax(y_pred[:,:,18:-4], axis=-1)))
    return acc

def notproj(y_pred, y_true):
    #add to gt
    acc = tf.constant(0,dtype=tf.float32)
    return acc

def Accuracy_proj(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for
    multiclass classification problems.
    '''
    result = tf.cond(tf.equal(tf.shape(y_true)[2],36), lambda: proj(y_pred, y_true), lambda: notproj(y_pred, y_true))
    return result

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss1 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss2 = SSDLoss(neg_pos_ratio=3, alpha=1.0)

losses = {
    "predictions_1": ssd_loss1.compute_loss,
    "predictions_2": ssd_loss2.compute_loss
}
lossWeights = {"predictions_1": 1.0,"predictions_2": 1.0}

model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights, metrics=[Accuracy]) 
# model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights) 


/scratch/tf3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Output "predictions_1_proj" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "predictions_1_proj" during training.
/scratch/tf3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Output "predictions_2_proj" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "predictions_2_proj" during training.


def accy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for
    multiclass classification problems.
    '''
    return K.mean(K.equal(K.argmax(y_true[:,:,:-4], axis=-1),
                  K.argmax(y_pred[:,:,:-4], axis=-1)))

def accy_proj(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for
    multiclass classification problems.
    '''
    return K.mean(K.equal(K.argmax(y_true[:,:,:14], axis=-1),
                  K.argmax(y_pred[:,:,:14], axis=-1)))

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss1 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss2 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
ssd_loss3 = SSDLoss_proj(neg_pos_ratio=3, alpha=1.0)

losses = {
    "predictions_1": ssd_loss1.compute_loss,
    "predictions_2": ssd_loss2.compute_loss,
    "predictions_1_proj_tot": ssd_loss3.compute_loss
}
lossWeights = {"predictions_1": 1.0,"predictions_2": 1.0,"predictions_1_proj_tot": 1.0}

# model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights, metrics=[accy,accy_proj]) 
model.compile(optimizer=adam, loss=losses, loss_weights=lossWeights) 


In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 600, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 300, 600, 3)  0                                            
__________________________________________________________________________________________________
identity_layer__1 (Lambda)      (None, 300, 600, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
identity_layer__2 (Lambda)      (None, 300, 600, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
input_mean

### 2.2 Load a previously created model


In [7]:
# train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path='dataset_pascal_voc_07+12_trainval.h5')
# val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path='dataset_pascal_voc_07_test.h5')
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset_1 = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)


VOC_2007_images_dir      = '../datasets/Images/'

# The directories that contain the annotations.
VOC_2007_annotations_dir      = '../datasets/VOC/Pasadena/Annotations/'

# The paths to the image sets.
VOC_2007_trainval_image_set_filename = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/trainval_sia.txt'
VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/val_sia.txt'
VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/test_sia.txt'

# VOC_2007_trainval_image_set_filename = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/trainval_sia_same.txt'
# VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/val_sia_same.txt'
# VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/test_sia_same.txt'

VOC_2007_trainval_image_set_filename = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/trainval_sia_sub.txt'
VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/val_sia_sub.txt'
VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/test_sia_sub.txt'
# VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/val_sia_sub.txt'
# VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/test_sia_sub.txt'

# VOC_2007_trainval_image_set_filename = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/trainval_one.txt'
# VOC_2007_val_image_set_filename      = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/val_one.txt'
# VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/test_one.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background',
           'tree']

train_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                        image_set_filenames=[VOC_2007_trainval_image_set_filename],
                        annotations_dirs=[VOC_2007_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)


val_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                      image_set_filenames=[VOC_2007_val_image_set_filename],
                      annotations_dirs=[VOC_2007_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=True,
                      ret=False)



Processing image set 'val_sia_sub.txt': 100%|██████████| 27/27 [00:00<00:00, 44.19it/s]


In [8]:
batch_size = 1

ssd_data_augmentation = SSDDataAugmentation_modified(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels_Modified()  
resize = Resize_Modified(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf__1').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf__1').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf__1').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=False,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()

val_dataset_size   = val_dataset.get_dataset_size()
,
print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size)) 

Number of images in the training dataset:	   133
Number of images in the validation dataset:	    27


## 4. Set the remaining training parameters

We've already chosen an optimizer and set the batch size above, now let's set the remaining training parameters. I'll set one epoch to consist of 1,000 training steps. The next code cell defines a learning rate schedule that replicates the learning rate schedule of the original Caffe implementation for the training of the SSD300 Pascal VOC "07+12" model. That model was trained for 120,000 steps with a learning rate of 0.001 for the first 80,000 steps, 0.0001 for the next 20,000 steps, and 0.00001 for the last 20,000 steps. If you're training on a different dataset, define the learning rate schedule however you see fit.

I'll set only a few essential Keras callbacks below, feel free to add more callbacks if you want TensorBoard summaries or whatever. We obviously need the learning rate scheduler and we want to save the best models during the training. It also makes sense to continuously stream our training history to a CSV log file after every epoch, because if we didn't do that, in case the training terminates with an exception at some point or if the kernel of this Jupyter notebook dies for some reason or anything like that happens, we would lose the entire history for the trained epochs. Finally, we'll also add a callback that makes sure that the training terminates if the loss becomes `NaN`. Depending on the optimizer you use, it can happen that the loss becomes `NaN` during the first iterations of the training. In later iterations it's less of a risk. For example, I've never seen a `NaN` loss when I trained SSD using an Adam optimizer, but I've seen a `NaN` loss a couple of times during the very first couple of hundred training steps of training a new model when I used an SGD optimizer.

In [9]:
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001

class prediction_history(Callback):
    def __init__(self):
        print("Predictor")
    def on_epoch_end(self, epoch, logs={}):
        ssd_loss1 = SSDLoss(neg_pos_ratio=3, alpha=1.0)
        predder = np.load('outputs/predder.npy')
        bX = predder[0][0]
        bZ = predder[0][1]
        gX = predder[0][2]
        gZ = predder[0][3]

        intermediate_layer_model = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_1").output)
        intermediate_layer_model_1 = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_1_proj_tot").output)
        intermediate_layer_model_2 = Model(inputs=model.input,
        intermediate_layer_model_3 = Model(inputs=model.input,
                             outputs=model.get_layer("predictions_2_proj_tot").output)

        intermediate_output = intermediate_layer_model.predict([bX,bZ,gX,gZ])
        intermediate_output_1 = intermediate_layer_model_1.predict([bX,bZ,gX,gZ])
        intermediate_output_2 = intermediate_layer_model_2.predict([bX,bZ,gX,gZ])
        intermediate_output_3 = intermediate_layer_model_3.predict([bX,bZ,gX,gZ])

        np.save('outputs/predictions_1_'+str(epoch)+'.npy',intermediate_output)
        np.save('outputs/predictions_1_proj_'+str(epoch)+'.npy',intermediate_output_1)
        np.save('outputs/predictions_2_'+str(epoch)+'.npy',intermediate_output_2)
        np.save('outputs/predictions_2_proj_'+str(epoch)+'.npy',intermediate_output_3)

In [10]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='checkpoints/double_ssd300_pascal_07+12_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
#model_checkpoint.best = 
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

csv_logger = CSVLogger(filename='ssd300_pascal_07+12_training_log.csv',
                       separator=',',
                       append=True)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule)

early_stopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

terminate_on_nan = TerminateOnNaN()
# printer_callback = prediction_history()
# custom_los = custom_loss()
callbacks = [
            model_checkpoint,
#             csv_logger,
#             custom_los,
            learning_rate_scheduler,
            early_stopping,
            terminate_on_nan,
#             printer_callback,
            tbCallBack
            ]

## 5. Train

In [11]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 500
steps_per_epoch = 1000

# history = model.fit_generator(generator=train_generator,
#                               steps_per_epoch=ceil(train_dataset_size/batch_size),
#                               epochs=final_epoch,
#                               callbacks=callbacks,
#                               verbose=1,
#                               validation_data=val_generator,
#                               validation_steps=ceil(val_dataset_size/batch_size),
#                               initial_epoch=initial_epoch)

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=ceil(train_dataset_size/batch_size),
                              epochs=final_epoch,
                              callbacks=callbacks,
                              verbose=1,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/500
y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  6.8333333333333335e-01
   4.4666666666666666e-01  1.6999999999999993e-01  3.2000000000000001e-01
   2.8353000000000000e+04  7.0000000000000000e+00  3.4155287117602207e+01
  -1.1816711173517416e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  8.9833333333333343e-01
   5.3500000000000003e-01  2.3333333333333317e-02  9.6666666666666679e-02
   9.9000000000000000e+01  7.0000000000000000e+00  3.4155287117602207e+01
  -1.1816711173517416e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  9.4833333333333325e-01
   4.9666666666666670e-01  5.3333333333333344e-02  1.1333333333333334e-01
   9.9000000000000000e+01  7.0000000000000000e+00  3.4155287117602207e+01
  -1.1816711173517416e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  9.7583333333333333e-01
   4.8333333333333328e-01  2.1666666666666723e-02  8.6666666666666614e-02
   9.9000000000000000e+01  7.0000000000

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  8.6499999999999999e-01
   4.7166666666666668e-01  1.4333333333333331e-01  1.9000000000000000e-01
   9.9000000000000000e+01  4.0000000000000000e+00  3.4142647101970169e+01
  -1.1813500060965308e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  7.7583333333333337e-01
   5.0333333333333341e-01  1.4833333333333332e-01  1.4000000000000001e-01
   9.9000000000000000e+01  4.0000000000000000e+00  3.4142647101970169e+01
  -1.1813500060965308e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  6.5666666666666673e-01
   5.1500000000000001e-01  5.6666666666666754e-02  8.3333333333333315e-02
   9.9000000000000000e+01  4.0000000000000000e+00  3.4142647101970169e+01
  -1.1813500060965308e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  2.8666666666666668e-01
   5.4166666666666674e-01  1.1000000000000001e-01  6.9999999999999951e-02
   9.9000000000000000e+01  4.0000000000000000e+00  

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  3.0833333333333335e-01
   3.5499999999999998e-01  1.0333333333333333e-01  2.4333333333333335e-01
   9.9000000000000000e+01  1.3000000000000000e+01  3.4144456584418471e+01
  -1.1814951000954494e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  1.3166666666666665e-01
   3.3499999999999996e-01  2.0666666666666667e-01  4.0333333333333332e-01
   9.9000000000000000e+01  1.3000000000000000e+01  3.4144456584418471e+01
  -1.1814951000954494e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  4.5250000000000001e-01
   2.7833333333333332e-01  3.4833333333333338e-01  5.5666666666666664e-01
   4.1074000000000000e+04  1.3000000000000000e+01  3.4144456584418471e+01
  -1.1814951000954494e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  8.1250000000000000e-01
   3.3666666666666667e-01  3.7166666666666659e-01  6.7333333333333334e-01
   9.9000000000000000e+01  1.3000000000000000e+01  

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  5.7583333333333331e-01
   3.8666666666666666e-01  6.4999999999999947e-02  2.5333333333333330e-01
   9.9000000000000000e+01  8.0000000000000000e+00  3.4153173366103509e+01
  -1.1811998991104326e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  3.8166666666666665e-01
   4.3333333333333335e-01  1.2000000000000000e-01  2.8666666666666668e-01
   9.9000000000000000e+01  8.0000000000000000e+00  3.4153173366103509e+01
  -1.1811998991104326e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  1.4583333333333334e-01
   3.6500000000000005e-01  4.4999999999999998e-02  3.9000000000000001e-01
   9.9000000000000000e+01  8.0000000000000000e+00  3.4153173366103509e+01
  -1.1811998991104326e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  1.3833333333333334e-01
   4.3333333333333335e-01  1.9666666666666666e-01  3.4666666666666668e-01
   9.9000000000000000e+01  8.0000000000000000e+00  

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  8.0000000000000002e-02
   4.7833333333333333e-01  1.9999999999999990e-02  1.5666666666666662e-01
   9.9000000000000000e+01  7.0000000000000000e+00  3.4126710148219672e+01
  -1.1816322632415705e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  1.1500000000000000e-01
   4.6333333333333337e-01  2.9999999999999999e-02  1.9333333333333341e-01
   9.9000000000000000e+01  7.0000000000000000e+00  3.4126710148219672e+01
  -1.1816322632415705e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  2.1833333333333332e-01
   4.4166666666666665e-01  1.3999999999999999e-01  3.1666666666666665e-01
   4.1601000000000000e+04  7.0000000000000000e+00  3.4126710148219672e+01
  -1.1816322632415705e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  3.6916666666666664e-01
   4.1666666666666669e-01  5.1666666666666694e-02  2.8666666666666674e-01
   9.9000000000000000e+01  7.0000000000000000e+00  

...

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  5.2583333333333337e-01
   4.5333333333333331e-01  8.1666666666666665e-02  1.8666666666666665e-01
   9.9000000000000000e+01  2.3000000000000000e+01  3.4150855083156827e+01
  -1.1814613665430080e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  6.2749999999999995e-01
   5.0333333333333341e-01  4.5000000000000040e-02  1.4000000000000001e-01
   9.9000000000000000e+01  2.3000000000000000e+01  3.4150855083156827e+01
  -1.1814613665430080e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  6.8250000000000000e-01
   5.1666666666666661e-01  5.4999999999999938e-02  1.6000000000000003e-01
   9.9000000000000000e+01  2.3000000000000000e+01  3.4150855083156827e+01
  -1.1814613665430080e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  7.4583333333333335e-01
   5.2333333333333332e-01  5.8333333333333348e-02  1.6666666666666669e-01
   9.9000000000000000e+01  2.3000000000000000e+01  

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  7.4500000000000000e-01
   4.4166666666666665e-01  1.3999999999999990e-01  3.7000000000000005e-01
   9.9000000000000000e+01  1.0000000000000000e+01  3.4152967925166529e+01
  -1.1811608503685102e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  9.0249999999999997e-01
   4.6166666666666667e-01  1.0499999999999998e-01  2.3666666666666664e-01
   4.2642000000000000e+04  1.0000000000000000e+01  3.4152967925166529e+01
  -1.1811608503685102e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  1.6166666666666668e-01
   3.8666666666666666e-01  7.6666666666666675e-02  1.8666666666666665e-01
   9.9000000000000000e+01  1.0000000000000000e+01  3.4152967925166529e+01
  -1.1811608503685102e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  2.2833333333333333e-01
   3.0666666666666664e-01  1.4000000000000001e-01  3.6666666666666664e-01
   9.9000000000000000e+01  1.0000000000000000e+01  

y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  4.2249999999999999e-01
   4.2500000000000004e-01  1.0499999999999998e-01  1.8333333333333340e-01
   9.9000000000000000e+01  2.0000000000000000e+01  3.4150590760855827e+01
  -1.1813413767183710e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  3.0916666666666670e-01
   3.9166666666666666e-01  1.1833333333333335e-01  3.2333333333333336e-01
   9.9000000000000000e+01  2.0000000000000000e+01  3.4150590760855827e+01
  -1.1813413767183710e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  2.0500000000000002e-01
   3.8166666666666665e-01  1.0666666666666669e-01  3.7666666666666659e-01
   9.9000000000000000e+01  2.0000000000000000e+01  3.4150590760855827e+01
  -1.1813413767183710e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  1.0416666666666667e-01
   4.2500000000000004e-01  7.8333333333333338e-02  2.5666666666666665e-01
   3.2919000000000000e+04  2.0000000000000000e+01  

133/133 [==============================] - 83s 626ms/step - loss: 105.3514 - predictions_1_loss: 40.4484 - predictions_2_loss: 49.7358 - predictions_1_Accuracy: 0.0313 - predictions_2_Accuracy: 0.0345 - val_loss: 28.2667 - val_predictions_1_loss: 6.0959 - val_predictions_2_loss: 7.6161 - val_predictions_1_Accuracy: 0.0188 - val_predictions_2_Accuracy: 0.0135

Epoch 00001: val_loss improved from inf to 28.26670, saving model to checkpoints/double_ssd300_pascal_07+12_epoch-01_loss-105.3514_val_loss-28.2667.h5
y_encoding_template:  (1, 17292, 18)
labels_one_hot:  [[ 0.0000000000000000e+00  1.0000000000000000e+00  6.8333333333333335e-01
   4.4666666666666666e-01  1.6999999999999993e-01  3.2000000000000001e-01
   2.8353000000000000e+04  7.0000000000000000e+00  3.4155287117602207e+01
  -1.1816711173517416e+02]
 [ 0.0000000000000000e+00  1.0000000000000000e+00  8.9833333333333343e-01
   5.3500000000000003e-01  2.3333333333333317e-02  9.6666666666666679e-02
   9.9000000000000000e+01  7.0000000

OSError: Unable to create file (unable to open file: name = 'checkpoints/double_ssd300_pascal_07+12_epoch-01_loss-105.3514_val_loss-28.2667.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)